In [1]:
import pandas as pd
import numpy as np
import hopsworks
import os 
# from dotenv import load_dotenv
import requests
import json
import datetime
import urllib.request
import codecs
import csv
import sys
from functions import *

import pandas as pd

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import f1_score
import xgboost as xgb

import warnings
warnings.filterwarnings("ignore")

In [2]:
import hopsworks

project = hopsworks.login() 

fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5359
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
feature_view = fs.get_feature_view(
    name = 'air_quality_fv',
    version = 6
)

In [4]:
# feature_view.create_training_data()

Training dataset job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/5359/jobs/named/air_quality_fv_6_1_create_fv_td_14012023150454/executions


(1, <hsfs.core.job.Job at 0x7fabacf2cac0>)

In [5]:
training_data = feature_view.get_training_data(1)[0]
training_data.head(20)

,aqi,iaqi_h,iaqi_p,iaqi_pm10,iaqi_t,date,o3_avg,o3_max,o3_min,pm10_avg,...,windgust,windspeed,winddir,pressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,conditions
0,21,1.516856,-2.813392,-1.482672,0.457763,1673136000000,2.481357,1.083473,2.514949,-2.339928,...,2.785763,2.387533,0.515942,-2.561441,1.491590,-0.387544,-2.396549,-2.393221,-1.872658,2
1,72,0.890074,0.457446,0.639196,0.521850,1673654400000,-0.610596,-1.083473,-0.295182,0.649486,...,-0.287413,-0.139512,0.378750,0.164496,-0.140725,0.510489,0.669104,0.719064,0.585206,0
2,34,0.024517,-0.257191,-0.652376,0.521850,1672552800000,0.116923,-0.120386,1.688440,0.524927,...,-0.170215,-0.330803,-1.284711,-0.728483,0.143431,-1.210741,0.257899,0.257985,1.404494,0
3,38,-0.273951,1.336999,-0.560120,-1.304625,1673481600000,-0.610596,0.842701,-0.295182,-0.222427,...,-0.287413,-0.371074,1.378541,0.845981,0.920122,0.996924,0.702536,0.680641,0.585206,0
4,46,-1.415590,-0.504565,2.299788,0.714110,1672725600000,0.116923,-2.046560,0.696629,-0.969780,...,-0.072551,-0.451618,-1.433051,0.751983,-0.488026,-1.248159,0.518663,0.526948,0.585206,0
5,25,1.143771,0.594876,-1.021396,-0.343322,1673395200000,-0.792475,-0.361158,-0.625785,0.524927,...,-0.450187,-0.310667,1.057853,0.540488,0.885392,0.510489,-0.427443,-0.472057,-1.053370,1
6,53,-1.990140,0.402474,-0.006590,2.124020,1672639200000,0.480682,0.842701,-0.129880,-0.471544,...,-0.612961,0.212864,0.195255,-0.610986,0.961167,-1.248159,-0.771786,-0.741020,-1.053370,1
7,65,-0.721652,0.017670,0.362431,-0.535583,1672812000000,-0.974355,-0.601929,-0.625785,1.272280,...,-0.710625,-0.481821,1.505444,1.574464,-1.520457,-1.248159,0.050625,0.027445,0.585206,3
8,78,-0.124717,0.072642,0.823706,-0.343322,1672898400000,-0.428716,0.601929,-0.791087,0.275809,...,-0.639005,-0.652976,-1.392750,0.681484,-1.593075,-0.799143,0.685820,0.680641,0.585206,3
9,78,0.076749,0.952195,0.639196,-1.400755,1673568000000,-0.974355,-0.120386,-0.791087,0.774045,...,-0.450187,-0.532161,-1.393608,0.446490,0.588608,0.360817,0.675791,0.680641,0.585206,0


In [6]:
training_data = training_data.sort_values(by=["date"], ascending=[False]).reset_index(drop=True)
training_data["aqi_next_day"] = training_data['aqi'].shift(1)

training_data

,aqi,iaqi_h,iaqi_p,iaqi_pm10,iaqi_t,date,o3_avg,o3_max,o3_min,pm10_avg,...,windspeed,winddir,pressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,conditions,aqi_next_day
0,72,0.890074,0.457446,0.639196,0.521850,1673654400000,-0.610596,-1.083473,-0.295182,0.649486,...,-0.139512,0.378750,0.164496,-0.140725,0.510489,0.669104,0.719064,0.585206,0,None
1,78,0.076749,0.952195,0.639196,-1.400755,1673568000000,-0.974355,-0.120386,-0.791087,0.774045,...,-0.532161,-1.393608,0.446490,0.588608,0.360817,0.675791,0.680641,0.585206,0,72
2,38,-0.273951,1.336999,-0.560120,-1.304625,1673481600000,-0.610596,0.842701,-0.295182,-0.222427,...,-0.371074,1.378541,0.845981,0.920122,0.996924,0.702536,0.680641,0.585206,0,78
3,25,1.143771,0.594876,-1.021396,-0.343322,1673395200000,-0.792475,-0.361158,-0.625785,0.524927,...,-0.310667,1.057853,0.540488,0.885392,0.510489,-0.427443,-0.472057,-1.053370,1,38
4,53,0.263291,1.089625,0.085665,-0.439452,1673308800000,0.116923,0.120386,-0.295182,0.151250,...,-0.572433,0.026336,0.634485,-0.721665,1.408522,0.689163,0.642218,0.585206,0,25
5,21,-0.796269,-0.971828,-1.482672,1.162718,1673222400000,1.753838,1.805788,0.696629,-1.218898,...,2.397601,0.786898,-0.939978,-0.434352,1.408522,0.518663,0.565371,-0.234082,1,53
6,21,1.516856,-2.813392,-1.482672,0.457763,1673136000000,2.481357,1.083473,2.514949,-2.339928,...,2.387533,0.515942,-2.561441,1.491590,-0.387544,-2.396549,-2.393221,-1.872658,2,21
7,85,1.516856,-0.449593,0.823706,-1.528928,1673049600000,0.298802,0.361158,-0.956389,-0.471544,...,-0.542229,-0.205177,-0.704984,1.185334,-0.237872,-1.971971,-1.970565,-1.872658,1,21
8,34,-0.109794,0.072642,-0.467865,0.393676,1672963200000,-0.974355,-1.324244,-0.791087,1.521398,...,-0.612704,-0.135723,-0.093998,-1.277346,1.184014,0.799486,0.795911,0.585206,3,85
9,78,-0.124717,0.072642,0.823706,-0.343322,1672898400000,-0.428716,0.601929,-0.791087,0.275809,...,-0.652976,-1.392750,0.681484,-1.593075,-0.799143,0.685820,0.680641,0.585206,3,34


In [7]:
training_data = training_data.iloc[1:]

In [8]:
X = training_data.drop(columns=["date"]).fillna(0)
y = training_data["aqi_next_day"]

In [9]:
model = xgb.XGBRegressor(n_estimators=10000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)

In [10]:
model.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.8,
             early_stopping_rounds=None, enable_categorical=False, eta=0.1,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.100000001, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=7, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=10000, n_jobs=0,
             num_parallel_tree=1, predictor='auto', ...)

In [11]:
y_pred = model.predict(X)
y_pred = np.array([int(x) for x in y_pred])

In [12]:
f1score = f1_score(y.astype(int), y_pred, average="micro")

In [13]:
f1score

0.46153846153846156

In [14]:
pred_comparison_df = pd.DataFrame({
    "aqi_true": y.astype(int),
    "aqi_pred": y_pred

})

pred_comparison_df

,aqi_true,aqi_pred
1,72,72
2,78,77
3,38,38
4,25,25
5,53,53
6,21,20
7,21,20
8,85,84
9,34,33
10,78,78


# Model Registry

In [15]:
mr = project.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


In [16]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X)
output_schema = Schema(y)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

model_schema.to_dict()

{'input_schema': {'columnar_schema': [{'name': 'aqi', 'type': 'int64'},
   {'name': 'iaqi_h', 'type': 'float64'},
   {'name': 'iaqi_p', 'type': 'float64'},
   {'name': 'iaqi_pm10', 'type': 'float64'},
   {'name': 'iaqi_t', 'type': 'float64'},
   {'name': 'o3_avg', 'type': 'float64'},
   {'name': 'o3_max', 'type': 'float64'},
   {'name': 'o3_min', 'type': 'float64'},
   {'name': 'pm10_avg', 'type': 'float64'},
   {'name': 'pm10_max', 'type': 'float64'},
   {'name': 'pm10_min', 'type': 'float64'},
   {'name': 'pm25_avg', 'type': 'float64'},
   {'name': 'pm25_max', 'type': 'float64'},
   {'name': 'pm25_min', 'type': 'float64'},
   {'name': 'tempmax', 'type': 'float64'},
   {'name': 'tempmin', 'type': 'float64'},
   {'name': 'temp', 'type': 'float64'},
   {'name': 'feelslikemax', 'type': 'float64'},
   {'name': 'feelslikemin', 'type': 'float64'},
   {'name': 'feelslike', 'type': 'float64'},
   {'name': 'dew', 'type': 'float64'},
   {'name': 'humidity', 'type': 'float64'},
   {'name': 'prec

In [17]:
import joblib

joblib.dump(model, "model.pkl")

['model.pkl']

In [18]:
model_reg = mr.python.create_model(
    name="xgboost_model",
    metrics={"f1": f1score},
    description="XGB Regressor for Air quality",
    input_example=X.sample().to_numpy(),
    model_schema=model_schema
)

model_reg.save('model.pkl')

  0%|          | 0/6 [00:00<?, ?it/s]

Model created, explore it at https://c.app.hopsworks.ai:443/p/5359/models/xgboost_model/3


Model(name: 'xgboost_model', version: 3)